In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase

tavily_client = TavilyClient()

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")


@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [4]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role: str = "external"

In [5]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Dynamically call tools based on the runtime context"""

    user_role = request.runtime.context.user_role
    
    if user_role == "internal":
        pass # internal users get access to all tools
    else:
        tools = [web_search] # external users only get access to web search
        request = request.override(tools=tools) 

    return handler(request)

In [7]:
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature=0.5)  

agent = create_agent(
    #model="gpt-5-nano",
    model = model,
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [9]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "internal"}
)

print(response["messages"][-1].content)

[{'type': 'text', 'text': 'There are 275 artists in the database.', 'extras': {'signature': 'Cs0BAXLI2nxO0YkTj9s8EptF9iXub5JXlOhTxyeu/jQBMmZznodhUjZAmzC2mPywjVwgfEBW+L6sHJo231otfiBfJ4AQZGDOiZ6u6AhKuQN/vksUxX+8oI5M+oKq1mC63w1GW7BqJpnPxGjg3Dc7n3tX5UnIzn819wB5VEhIeIF3BnRbtct4y/8F20wVgkdv9X5vthTxhERR2ipH5+YwklD7QSgiQmvZCscwBxzzhtjc6FwyllgDHXs2s2YowndGKLm76VSLgd9OwpHr2Q7+VQ=='}}]
